In this homework, we'll deploy the bees vs wasps model we trained in the previous homework.

In [1]:
import subprocess
import os
from pathlib import Path
from io import BytesIO
from urllib import request
from PIL import Image

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.lite as tflite

In [4]:
data_file_url = 'https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5'
path = Path.cwd() / 'bees-wasps.h5'
if not path.is_file():
    subprocess.run(['wget', '-c', data_file_url])

### Question 1

Now convert this model from Keras to TF-Lite format. What's the size of the converted model?

* 21 Mb
* 43 Mb
* 80 Mb
* 164 Mb

In [5]:
model = keras.models.load_model('bees-wasps.h5', compile=False)

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/47/qjky30_d7_gbx41qhtyr_0z80000gn/T/tmpsh9q5pq3/assets


INFO:tensorflow:Assets written to: /var/folders/47/qjky30_d7_gbx41qhtyr_0z80000gn/T/tmpsh9q5pq3/assets


In [7]:
with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

### Question 2

To be able to use this model, we need to know the index of the input and the index of the output. What's the output index for this model?

* 3
* 7
* 13
* 24

In [8]:
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


In [9]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
print(f'Input index: {input_index}, Output index: {output_index}')

Input index: 0, Output index: 13
